In [1]:
'''
Notebook to test all PMU functions:
- connect to the PMU
- ask for the CFG2 frame
- decode the CFG2 frame
- send command to the PMU
- ask for dataframe
- decode dataframe
- create a PMU frequency plot.

It needs my own functions:
- get_crc16.py
- read_data.py
- request_cfg2.py
- send_command.py
- decode_cfg2.py
- decode_dataframe.py
'''

'\nNotebook to test all PMU functions:\n- connect to the PMU\n- ask for the CFG2 frame\n- decode the CFG2 frame\n- send command to the PMU\n- ask for dataframe\n- decode dataframe\n- create a PMU frequency plot.\n\nIt needs my own functions:\n- get_crc16.py\n- read_data.py\n- request_cfg2.py\n- send_command.py\n- decode_cfg2.py\n- decode_dataframe.py\n'

In [8]:
import socket
import time
import math
import crcmod
from get_crc16 import get_crc16 #own function
from request_cfg2 import request_cfg2 #own function
from decode_cfg2 import decode_cfg2 #own function
from send_command import send_command #own function
from decode_dataframe import decode_dataframe #own function
from read_data import read_data #own function
import matplotlib.pyplot as plt
from drawnow import drawnow
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Text output

In [2]:
#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)
#decode cfg2 and get parameters of the pmu in a dict
diccfg2=decode_cfg2(cfg2)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)


#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for n dataframes
for i in range(5):
    [dframe,crcok] = read_data(s)
    print('Dataframe # {}, CRC OK: {}:-----------------------------------------'.format(i+1,crcok))
#     print(dframe.hex())
    #Important: if the crc of the dataframe is wrong, is better to not
    #decode data because magnitudes of voltage could be wrong and cause
    #trouble, also, the function decode_dataframe is tested only
    #with good dataframes
    if crcok:
        dictdf = decode_dataframe(dframe,diccfg2,verbose=False)
        print('ACK byte: {}'.format(dictdf['ACK']))
        print('Frame type: {}'.format(dictdf['frameType']))
        print('Frame type string: {}'.format(dictdf['frameTypeStr']))
        print('Protocol version: {}'.format(dictdf['protocolVer']))
        print('Framesize: {}'.format(dictdf['FRAMESIZE']))
        print('IDCODE: {}'.format(dictdf['IDCODE']))
        print('SOC: {}'.format(dictdf['SOC']))
        print('FRACSEC raw: {}'.format(dictdf['FRACSEC']))
        print('TIME_QUALITY raw: {}'.format(dictdf['TIME_QUALITY']))
        print('Unix Time: {}'.format(dictdf['TIME_UNIX']))
        print('Human readable time: {}'.format(dictdf['TIME']))
        print('Stat bits raw: {}'.format(dictdf['STAT_bits'].hex()))
        print('Data is valid: {}'.format(dictdf['STAT_VALID_DATA']))
        print('PMU error: {}'.format(dictdf['STAT_PMU_ERROR']))
        print('PMU sync: {}'.format(dictdf['STAT_TIME_SYNC']))
        print('Data sorting: {}'.format(dictdf['STAT_DATA_SORTING']))
        print('PMU trigger detected: {}'.format(dictdf['STAT_TRIGGER_DETECTED']))
        print('Configuration changed: {}'.format(dictdf['STAT_CONFIG_CHANGED']))
        print('Unlocked time: {}'.format(dictdf['STAT_UNLOCKED_TIME']))
        print('Trigger reason: {}'.format(dictdf['STAT_TRIGGER_REASON']))
        for i in range(diccfg2['PHNMR']):
            print('Phasor #{}: {}, {:0.2f}{} {:0.2f}°'.format(
                i+1,diccfg2['PHASOR_names'][i],dictdf['PHASORS_magnitude'][i],
                    diccfg2['PHUNIT_str'][i][0],dictdf['PHASORS_angle'][i]))

        for i,n in enumerate(diccfg2['ANALOG_names']):
            print('Analog value #{}: {}, {:0.3f}'.format(
                i+1,n,round(dictdf['ANALOG'][i],3)))
        print('Frequency: {:0.6f}'.format(dictdf['FREQ']))
        print('DFREQ: {:0.6f}'.format(dictdf['DFREC']))
        print('Digital word: 0x{}'.format(dictdf['DIGITAL'].hex()))
    else:
        print('Data trasnmission error:\n {}'.format(dframe))
    print('')
    
#send command to turn off data transmission
cmd = 0b000
send_command(s,idcode,cmd)
s.close()

Dataframe # 1, CRC OK: True:-----------------------------------------
ACK byte: 170
Frame type: 0
Frame type string: Data Frame
Protocol version: 1
Framesize: 124
IDCODE: 1
SOC: 72
FRACSEC raw: 4000000
TIME_QUALITY raw: 15
Unix Time: 72.4
Human readable time: 1970-01-01 00:01:12.400000
Stat bits raw: 20d0
Data is valid: True
PMU error: False
PMU sync: False
Data sorting: timestamp
PMU trigger detected: False
Configuration changed: False
Unlocked time: unlocked_10
Trigger reason: manual
Phasor #1: b'V2+             ', 0.29V -172.10°
Phasor #2: b'VA2             ', 0.87V -172.02°
Phasor #3: b'IA2             ', 0.01A 42.75°
Phasor #4: b'VB2             ', 0.00V 34.72°
Phasor #5: b'IB2             ', 0.01A 44.08°
Phasor #6: b'VC2             ', 0.00V 1.82°
Phasor #7: b'IC2             ', 0.00A 41.29°
Phasor #8: b'IN2             ', 0.00A -64.09°
Analog value #1: b'P1              ', 0.000
Analog value #2: b'S1              ', 0.000
Analog value #3: b'Q1              ', -0.000
Analog value

# Matplotlib output

In [6]:
def make_fig():
    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.plot(x, y)  

In [9]:
#matplotlib
#popup window
%matplotlib tk 
plt.ion()  # enable interactivity
fig = plt.figure()  # make a figure
plt.autoscale(enable=True)
x = list()
y = list()

#PMU idcode
idcode=1
tcpip= "10.10.200.22"
tcpport = 4712
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
s.connect((tcpip, tcpport))

#ask for cfg2 to get the PMU parameters to use with the received dataframe
cfg2 = request_cfg2(s,idcode)
#decode cfg2 and get parameters of the pmu in a dict
diccfg2=decode_cfg2(cfg2)

#send command to turn ON data transmission
cmd = 0b0010
send_command(s,idcode,cmd)

#read  measurements 
#this code is not optimal, n frames per second are expected
#is only for testing purposes, data could be lost or superimposed
n= 0
dataframenumber = 0 #counter for dataframes
#ask for n dataframes
for i in range(100):
    [dframe,crcok] = read_data(s)
#     print(dframe.hex())
    #Important: if the crc of the dataframe is wrong, is better to not
    #decode data because magnitudes of voltage could be wrong and cause
    #trouble, also, the function decode_dataframe is tested only
    #with good dataframes
    if crcok:
        dictdf = decode_dataframe(dframe,diccfg2,verbose=False)       
        x.append(pd.to_datetime(dictdf['TIME'], format = '%Y-%m-%d %H:%M:%S.%f'))
        y.append(dictdf['FREQ'])  # or any arbitrary update to your figure's data
        drawnow(make_fig)
        #print(dictdf['TIME'])
        if len(x) >30:
            x.pop(0)
            y.pop(0)
    else:
        print('Data transmission error: \n {}'.format(dframe))
    
#send command to turn off data transmission
cmd = 0b000
send_command(s,idcode,cmd)
s.close()